In [1]:
from manim import *
import numpy as np

class PCADeepDiveScene(ThreeDScene):
    def construct(self):
        # ----------------- SETTINGS -----------------
        self.set_camera_orientation(phi=75*DEGREES, theta=-60*DEGREES, distance=12)
        fps = 30

        # ----------------- DATASET -----------------
        def curve_func(t):
            return np.array([np.cos(t)*2, np.sin(t)*2, 0.8*np.sin(2*t)])  # slightly flattened z

        t_vals = np.linspace(0, 2*np.pi, 30)
        points = np.array([curve_func(t) for t in t_vals])

        # 3D axes + points
        axes = ThreeDAxes(
            x_range=[-5, 5, 1], y_range=[-5, 5, 1], z_range=[-3, 3, 1],
            x_length=8, y_length=8, z_length=6
        )
        self.add(axes)
        dots = VGroup(*[Dot3D(point=pt, color=BLUE, radius=0.08) for pt in points])
        self.play(FadeIn(dots), run_time=1.0)

        # ----------------- MEAN -----------------
        mean = points.mean(axis=0)
        mean_dot = Dot3D(point=mean, color=RED, radius=0.12)
        mean_label = MathTex(r"\bar{x} = (" + f"{mean[0]:.2f}," + f"{mean[1]:.2f}," + f"{mean[2]:.2f})").scale(0.6)
        mean_label.to_corner(UL)
        cov_formula = MathTex(r"\Sigma = \frac{1}{N}\sum_i (x_i-\bar{x})(x_i-\bar{x})^T").scale(0.55)
        cov_formula.next_to(mean_label, DOWN, aligned_edge=LEFT)
        self.add_fixed_in_frame_mobjects(mean_label, cov_formula)
        self.play(FadeIn(mean_dot), run_time=0.7)
        self.wait(0.3)

        # ----------------- COVARIANCE MATRIX (numeric) -----------------
        centered = points - mean
        cov = np.cov(centered.T, bias=True)  # 1/N normalization to match displayed formula
        cov_rounded = np.round(cov, 3)
        cov_matrix = Matrix([[f"{v:.3f}" for v in row] for row in cov_rounded], left_bracket="(", right_bracket=")")
        cov_matrix.scale(0.7).to_corner(UR).shift(LEFT*0.4 + DOWN*0.2)
        cov_label = MathTex(r"\Sigma").scale(0.7).next_to(cov_matrix, LEFT, buff=0.2)
        self.add_fixed_in_frame_mobjects(cov_matrix, cov_label)
        self.play(FadeIn(cov_matrix), FadeIn(cov_label), run_time=0.8)
        self.wait(0.6)

        # ----------------- EIGENDECOMPOSITION -----------------
        eigvals, eigvecs = np.linalg.eigh(cov)
        idx = np.argsort(eigvals)[::-1]
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
        # numeric Lambda matrix and Q matrix for display
        lambda_mat = np.diag(eigvals)
        lambda_disp = Matrix([[f"{v:.3f}" if i==j else "0.000" for j,vj in enumerate(lambda_mat[i])] for i in range(3)])
        q_disp = Matrix([[f"{v:.3f}" for v in eigvecs[i]] for i in range(3)])
        q_disp.scale(0.6).next_to(cov_matrix, DOWN, buff=0.4)
        lambda_disp.scale(0.6).next_to(q_disp, RIGHT, buff=0.2)
        qT_disp = Matrix([[f"{v:.3f}" for v in eigvecs[:,i]] for i in range(3)])  # transpose display (rows are columns)
        qT_disp.scale(0.6).next_to(lambda_disp, RIGHT, buff=0.2)
        eqn = MathTex(r"\Sigma = Q \Lambda Q^\top").scale(0.7).next_to(cov_matrix, LEFT, buff=0.2)
        self.add_fixed_in_frame_mobjects(eqn, q_disp, lambda_disp, qT_disp)
        self.play(FadeIn(eqn), FadeIn(q_disp), FadeIn(lambda_disp), FadeIn(qT_disp), run_time=1.0)
        self.wait(0.6)

        # ----------------- PRINCIPAL AXES (visual) -----------------
        # Normalize eigenvectors & create arrows length ~ sqrt(eigvals)
        eigvecs = eigvecs / np.linalg.norm(eigvecs, axis=0)
        pc_arrows = VGroup()
        colors = [YELLOW, GREEN, PURPLE]
        scale_factor = 2.2
        pc_labels = VGroup()
        for i in range(3):
            v = eigvecs[:, i]
            length = np.sqrt(max(eigvals[i], 0)) * scale_factor
            start = mean - v * length
            end = mean + v * length
            arr = Arrow3D(start=start, end=end, thickness=0.02, color=colors[i])
            pc_arrows.add(arr)
            lab = MathTex(fr"PC_{i+1}").scale(0.5)
            lab.move_to(mean + v*(length+0.45))
            self.add_fixed_orientation_mobjects(lab)
            pc_labels.add(lab)
        self.play(Create(pc_arrows), run_time=1.6)
        self.wait(0.4)

        # Show that arrow length is std-dev: add small note
        std_note = MathTex(r"\text{length} \propto \sqrt{\lambda_i}").scale(0.55)
        std_note.to_corner(DR)
        self.add_fixed_in_frame_mobjects(std_note)
        self.play(FadeIn(std_note))
        self.wait(0.6)

        # ----------------- EXPLAINED VARIANCE BARS -----------------
        explained = eigvals / eigvals.sum()
        cum_explained = np.cumsum(explained)
        # create bars as rectangles in fixed frame
        bars = VGroup()
        labels_bars = VGroup()
        max_h = 2.5
        for i, val in enumerate(explained):
            rect = Rectangle(width=0.6, height=val*max_h, fill_opacity=0.8, fill_color=colors[i], stroke_width=1)
            rect.next_to(ORIGIN, UP)  # temporarily
            rect.to_corner(UR).shift(LEFT*(i*0.9) + DOWN*1.3)
            num = MathTex(f"{val*100:.1f}\\%").scale(0.45)
            num.next_to(rect, DOWN, buff=0.07)
            bars.add(rect)
            labels_bars.add(num)
            # add fixed orientation/position
            self.add_fixed_in_frame_mobjects(rect, num)
        title_bars = MathTex(r"\text{Explained variance}").scale(0.6).next_to(bars, UP, buff=0.1)
        self.add_fixed_in_frame_mobjects(title_bars)
        self.play(FadeIn(*bars), FadeIn(*labels_bars), FadeIn(title_bars), run_time=1.0)
        self.wait(0.6)

        # show cumulative number
        cum_text = MathTex(fr"\text{{cumulative (top 2)}}={cum_explained[1]*100:.1f}\%").scale(0.55)
        cum_text.next_to(title_bars, DOWN, buff=0.1)
        self.add_fixed_in_frame_mobjects(cum_text)
        self.play(Write(cum_text))
        self.wait(0.8)

        # ----------------- PROJECTION & RECONSTRUCTION for k = 1..3 -----------------
        # helper: projection function
        def project_to_k(points_, mean_, eigvecs_, k_):
            Qk = eigvecs_[:, :k_]
            centered_ = points_ - mean_
            # coordinates in k-dim
            Ys = (Qk.T @ centered_.T).T       # shape N x k
            Xhat = (Qk @ Ys.T).T + mean_      # shape N x 3
            return Xhat, Ys

        # original points are currently at their positions (we will show reconstructions)
        recon_dots_group = VGroup()
        connector_lines_group = VGroup()

        # select a single sample point to do numeric walkthrough
        sample_idx = 3
        sample_pt = points[sample_idx]
        sample_dot = Dot3D(point=sample_pt, color=TEAL, radius=0.12)
        self.play(FadeIn(sample_dot))
        sample_label = MathTex(f"x_{{{sample_idx}}}=(" + f"{sample_pt[0]:.2f}," + f"{sample_pt[1]:.2f}," + f"{sample_pt[2]:.2f})").scale(0.55)
        sample_label.to_corner(LEFT)
        self.add_fixed_in_frame_mobjects(sample_label)
        self.play(Write(sample_label))
        self.wait(0.4)

        residual_texts = VGroup()
        for k in [1, 2, 3]:
            # compute reconstruction
            Xhat, Ys = project_to_k(points, mean, eigvecs, k)
            recon_dots = VGroup(*[Dot3D(point=pt, color=ORANGE, radius=0.07) for pt in Xhat])
            # connectors from recon to original
            connectors = VGroup(*[Line3D(start=points[i], end=Xhat[i], color=GRAY) for i in range(len(points))])

            # show connectors and reconstruction for current k
            self.play(LaggedStart(*[Create(c) for c in connectors], lag_ratio=0.01), run_time=0.9)
            self.play(LaggedStart(*[Create(rd) for rd in recon_dots], lag_ratio=0.01), run_time=0.6)
            # animate moving the sample dot's label to show its reconstruction numeric walk
            sample_proj = Xhat[sample_idx]
            # show numeric projection step for that sample
            # compute coordinates in PC basis
            Qk = eigvecs[:, :k]
            coord = (Qk.T @ (sample_pt - mean))
            recon_label = MathTex(
                rf"\hat{{x}}_{{{sample_idx}}} = \bar{{x}} + Q_{{{k}}} Q_{{{k}}}^T (x_{{{sample_idx}}}-\bar{{x}})"
            ).scale(0.5)
            recon_label.to_corner(LEFT).shift(DOWN*0.8)
            self.add_fixed_in_frame_mobjects(recon_label)
            numeric_label = MathTex(
                rf"Q_{{{k}}}^T(x-\bar{{x}})=\;(" + ", ".join([f"{v:.3f}" for v in coord]) + ")"
            ).scale(0.45)
            numeric_label.to_corner(LEFT).shift(DOWN*1.25)
            self.add_fixed_in_frame_mobjects(numeric_label)
            self.play(Write(recon_label), Write(numeric_label))
            # highlight the sample's reconstruction by animating a small larger dot at recon location
            highlight = Dot3D(point=sample_proj, color=YELLOW, radius=0.12)
            self.play(FadeIn(highlight), run_time=0.5)
            self.wait(0.6)

            # compute residual variance for this k (mean squared residual)
            residual_var_k = np.mean(np.linalg.norm(points - Xhat, axis=1)**2)
            res_text = MathTex(f"k={k}\\;\\text{{residual MSE}}={residual_var_k:.4f}").scale(0.5)
            res_text.to_corner(DL)
            self.add_fixed_in_frame_mobjects(res_text)
            self.play(Write(res_text))
            residual_texts.add(res_text)
            self.wait(0.8)

            # cleanup the recon objects & labels for next k (but keep the numeric residual historical)
            self.play(FadeOut(recon_dots), FadeOut(connectors), FadeOut(highlight))
            self.play(FadeOut(recon_label), FadeOut(numeric_label))
            self.wait(0.2)

        # keep residual numbers visible for comparison
        self.play(*[FadeIn(rt) for rt in residual_texts], run_time=0.6)
        self.wait(1.0)

        # ----------------- SVD NOTE (equivalence) -----------------
        svd_eq = MathTex(r"X_c = U \Sigma V^\top \quad\Rightarrow\quad V = Q").scale(0.6)
        svd_eq.to_corner(DR)
        self.add_fixed_in_frame_mobjects(svd_eq)
        self.play(Write(svd_eq))
        self.wait(0.8)

        # ----------------- CAMERA FOCUS: ZOOM into mean & plane -----------------
        self.play(self.camera.frame.animate.set(width=6).move_to(mean), run_time=1.0)
        # show plane for top-2 once again and zoom a little
        v1, v2 = eigvecs[:, 0], eigvecs[:, 1]
        plane_size = 3.2
        corners = [mean +  v1*plane_size +  v2*plane_size,
                   mean +  v1*plane_size - v2*plane_size,
                   mean -  v1*plane_size - v2*plane_size,
                   mean -  v1*plane_size + v2*plane_size]
        plane = Polygon(*corners, color=YELLOW, fill_opacity=0.25)
        self.play(FadeIn(plane), run_time=0.8)
        self.wait(0.6)

        # highlight orthogonality: show dot products of PC vectors ~ 0
        ortho_texts = VGroup()
        for i in range(3):
            for j in range(i+1,3):
                val = np.dot(eigvecs[:,i], eigvecs[:,j])
                tx = MathTex(f"v_{i+1} \\cdot v_{j+1} = {val:.3e}").scale(0.45)
                tx.next_to(plane, DOWN).shift(RIGHT*(0.8*(i)) + DOWN*(0.18*(j)))
                self.add_fixed_in_frame_mobjects(tx)
                ortho_texts.add(tx)
                self.play(Write(tx), run_time=0.2)
        self.wait(1.0)

        # ----------------- FINISH: ambient rotate and final note -----------------
        final_note = MathTex(r"\text{Project: } \hat{x} = \bar{x} + Q_k Q_k^\top (x-\bar{x})").scale(0.6)
        final_note.to_corner(UL).shift(RIGHT*1.0)
        self.add_fixed_in_frame_mobjects(final_note)
        self.play(Write(final_note))
        self.begin_ambient_camera_rotation(rate=0.02)
        self.wait(4.0)
        self.stop_ambient_camera_rotation()
        self.wait(0.5)

%manim -ql -v ERROR PCADeepDiveScene

Manim Community v0.19.0

NameError: cannot access free variable 'v' where it is not associated with a value in enclosing scope